# Load Libraries

In [1]:
!pip install openprompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install sentencepiece
!pip  install transformers
!pip install pytorch-ignite
!pip install datasets



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt import PromptDataLoader, PromptForClassification
from tqdm import tqdm
from transformers import AdamW, get_scheduler
from torch.nn import CrossEntropyLoss

In [4]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AdamW, get_linear_schedule_with_warmup
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import AutoTokenizer, AutoModel, AdamW
import random
import os
from urllib import request
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [5]:
import os
import sys
from datasets import load_metric

# Helper Function

In [6]:
from openprompt.data_utils import InputExample


metric_f1 = load_metric("f1")
metric_precision = load_metric("precision")
metric_recall = load_metric("recall")

class PromptUtil:

    # template = '{"placeholder":"text_a"} اللكنه هي {"mask"} .'
    # template = '{"placeholder":"text_a"}  .{"mask"} تحليل المشاعر'

    template = '{"placeholder":"text_a"}  .{"mask"} المشاعر'
    # verbalizer = {
    #     # 0: ["respectful", "friendly", "great"],
    #     # 1: ["patronizing", "condescending", "terrible"],
    #     0: ["respectful"],
    #     1: ["patronizing"],
    #     # 0: ["great"],
    #     # 1: ["terrible"],
    # }
    
    def data2examples(df):
        df = df.reset_index()
        examples = []
        for _, row in df.iterrows():
            examples.append(InputExample(text_a=row['text'], label=row["label"]))
        return examples
    
    def test2examples(df):
        df = df.reset_index()
        examples = []
        for _, row in df.iterrows():
            examples.append(InputExample(text_a=row['text']))
        return examples

In [7]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cuda',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

In [8]:
import logging
from logging.handlers import RotatingFileHandler
logger = logging.getLogger()

CRITICAL = logging.CRITICAL
ERROR = logging.ERROR
WARNING = logging.WARNING
WARN = WARNING
INFO = logging.INFO
DEBUG = logging.DEBUG

class context_disabled():
    def __enter__(self):
        disabled()
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        enabled()

def init_logger(log_file=None,
                log_file_level=logging.NOTSET,
                rotate=False,
                log_level=logging.INFO,):
    """
    Adopted from OpenNMT-py:
        https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/utils/logging.py
    """
    log_format = logging.Formatter("[%(asctime)s %(levelname)s] %(message)s")
    logger = logging.getLogger()
    logger.setLevel(log_level)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(log_format)
    logger.handlers = [console_handler]

    if log_file and log_file != '':
        if rotate:
            file_handler = RotatingFileHandler(
                log_file, maxBytes=1000000, backupCount=10)
        else:
            file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(log_file_level)
        file_handler.setFormatter(log_format)
        logger.addHandler(file_handler)

def disabled():
    return logging.disable(logging.CRITICAL)

def enabled():
    return logging.disable(logging.NOTSET)

# Begin Training

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
train = pd.read_csv('/content/drive/MyDrive/wanlp/Datasets/NADI2022-Train/NADI2022-Train/Subtask2/NADI2022_Subtask2_TRAIN.tsv', sep='\t', lineterminator='\n')
valid = pd.read_csv('/content/drive/MyDrive/wanlp/Datasets/NADI2022-Train/NADI2022-Train/Subtask2/NADI2022_Subtask2_DEV.tsv', sep='\t', lineterminator='\n')


In [11]:
train.columns

Index(['#1_id', '#2_content', '#3_label'], dtype='object')

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['#3_label'] = le.fit_transform(train['#3_label'].values)
valid['#3_label'] = le.transform(valid['#3_label'].values)
train.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)
valid.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)


In [13]:
# verbalizer = {i: [l] for (i, l) in enumerate(le.classes_)}


In [14]:
verbalizer ={2:'إيجابي',    
             0: 'سلبي',
             1:"محايد"}

In [15]:
train_batch_size = 8
val_batch_size = 8
num_epochs = 30
weight_decay = 1e-3
warmup_ratio = 0 # 0.1
lr = 1e-4
max_seq_length = 128
early_stop_steps = 25
model_name = "bert"
model_path = 'aubmindlab/bert-base-arabertv02-twitter'
exp_name = "prompt-tuning-electra"

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classes = [i for i in range (3)]

In [17]:
plm, tokenizer, model_config, WrapperClass = load_plm(model_name, model_path)
    
promptTemplate = ManualTemplate(
    text = PromptUtil.template,
    tokenizer = tokenizer,
)

In [18]:
train_dataset = PromptUtil.data2examples(train)

In [19]:
val_dataset = PromptUtil.data2examples(valid)

In [20]:
train_dataloader = PromptDataLoader(
        dataset = train_dataset,
        tokenizer = tokenizer,
        template = promptTemplate,
        tokenizer_wrapper_class=WrapperClass,
        batch_size = train_batch_size,
        shuffle = True,
        max_seq_length=max_seq_length,
        truncate_method="head",
    )
    
val_dataloader = PromptDataLoader(
    dataset = val_dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    batch_size = val_batch_size,
    max_seq_length=max_seq_length,
    truncate_method="head",
)

tokenizing: 1500it [00:01, 1126.95it/s]
tokenizing: 500it [00:00, 1103.40it/s]


In [21]:
promptVerbalizer = ManualVerbalizer(
        classes = classes,
        label_words = verbalizer,
        tokenizer = tokenizer,
    )

In [22]:
model = PromptForClassification(
        template = promptTemplate,
        plm = plm,
        verbalizer = promptVerbalizer,
    )
model.to(device)

PromptForClassification(
  (prompt_model): PromptModel(
    (plm): BertForMaskedLM(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(64000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
          

In [23]:
num_training_steps = num_epochs * len(train_dataloader)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
        'lr' : lr
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
        'lr' : 2e-4
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

In [24]:
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
    # lr_scheduler = get_constant_schedule(optimizer=optimizer)
loss_func = CrossEntropyLoss()

In [25]:
seed = 42

In [26]:
progress_bar = tqdm(range(num_training_steps))
best_f1 = 0
best_recall = 0
best_precision = 0
early_stop = 0

# num_epochs = num_training_steps // len(train_dataloader)
train_steps = 0
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(batch)
        loss = loss_func(logits, batch["label"])
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        progress_bar.set_postfix({'loss': loss.item(), "lr": lr_scheduler.get_last_lr()[0]})
        train_steps += 1
        
        # if train_steps % eval_step == 0:
    model.eval()
    predictions = []
    total_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(batch)
            total_loss += loss_func(logits, batch["label"])
            preds = torch.argmax(logits, dim = -1)
            predictions += preds.flatten().tolist()

    f1 = metric_f1.compute(predictions=predictions, references=valid['label'].values, average='macro')["f1"]
    precision = metric_precision.compute(predictions=predictions, references=valid['label'].values, average='macro')["precision"]
    recall = metric_recall.compute(predictions=predictions, references=valid['label'].values, average='macro')["recall"]

    logger.info(f"Epoch {epoch}: Loss = {total_loss/len(val_dataloader)}, F1 = {f1}, Precision = {precision}, Recall = {recall}")

    if best_f1 < f1:
        best_f1 = f1
        best_recall = recall
        best_precision = precision
        early_stop = 0
        torch.save(model.state_dict(), f"/content/drive/MyDrive/wanlp/NADI/sub_task_2_saved_models/aralectra_prompt/arabert_base_playaround_{seed}_{lr}_{epoch}_{f1:.2f}_{precision:.2f}_{recall:.2f}.pt")
    else:
        early_stop += 1
        if early_stop >= early_stop_steps:
            logger.info(f"Early stop at epoch {epoch}")
            break
    
logger.info(f"Final Result: F1 = {best_f1}, Precision = {best_recall}, Recall = {best_precision}")

 87%|████████▋ | 4888/5640 [25:36<03:07,  4.00it/s, loss=1.08e-5, lr=1.33e-5]